#### 本项目基于PaddleSeg实现

In [0]:
### 对遥感地块分割的训练集、划分训练集、验证集
import random
from pathlib import Path

root_path = Path(r'/Path/to/CCF_BCI_2020/data')

images_path = root_path/'img_train'
labels_path = root_path/'lab_train'

train_txt = open(root_path/'train.txt', 'w')
val_txt = open(root_path/'val.txt', 'w')

for label_path in labels_path.rglob('*.png'):
    if random.random() < 0.9:
        train_txt.write(f'{images_path}/{label_path.stem}.jpg {labels_path}/{label_path.name} \n')
    else:
        val_txt.write(f'{images_path}/{label_path.stem}.jpg {labels_path}/{label_path.name} \n')

train_txt.close()
val_txt.close()

In [ ]:
# _base_/ccf_bci.yml

# 存放对数据集以及优化器的操作

train_dataset:
  type: Dataset
  dataset_root: /home/ubuntu/workspace/Projects/CCF_BCI_2020/data
  train_path: /home/ubuntu/workspace/Projects/CCF_BCI_2020/data/train.txt
  num_classes: 4
  transforms:
    - type: ResizeStepScaling
      min_scale_factor: 0.5
      max_scale_factor: 2.0
      scale_step_size: 0.25
    - type: RandomPaddingCrop
      crop_size: [256, 256]
    - type: RandomHorizontalFlip
    - type: RandomDistort
      brightness_range: 0.4
      contrast_range: 0.4
      saturation_range: 0.4
    - type: Normalize
  mode: train

val_dataset:
  type: Dataset
  dataset_root: /home/ubuntu/workspace/Projects/CCF_BCI_2020/data
  val_path: /home/ubuntu/workspace/Projects/CCF_BCI_2020/data/val.txt
  num_classes: 4
  transforms:
    - type: Normalize
  mode: val


optimizer:
  type: sgd
  momentum: 0.9
  weight_decay: 4.0e-5

lr_scheduler:
  type: PolynomialDecay
  learning_rate: 0.001
  end_lr: 0
  power: 0.9

In [ ]:
# model.yml
_base_: '../_base_/ccf_bci.yml'

batch_size: 64
iters: 80000


optimizer:
  type: sgd
  momentum: 0.9
  weight_decay: 4.0e-5

lr_scheduler:
  type: PolynomialDecay
  learning_rate: 0.01
  end_lr: 0
  power: 0.9

loss:
  types:
    - type: MixedLoss
      losses:
        - type: CrossEntropyLoss
        - type: LovaszSoftmaxLoss
        - type: DiceLoss
      coef: [0.6, 0.1, 0.3]
  coef: [1]

model:
  type: FCN
  backbone:
    type: HRNet_W48
    align_corners: False
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/hrnet_w48_ssld.tar.gz
  backbone_indices: [-1]
  pretrained: Null

In [ ]:
# 项目运行
cd PaddleSeg

# 模型训练
export CUDA_VISIBLE_DEVICES=1 # 设置1张可用的卡
python train.py \
       --config configs/ccf_bci/model.yml \
       --do_eval \
       --use_vdl \
       --save_interval 500 \
       --save_dir output_loss_mixed_1 \

# 模型预测
python predict.py \
       --config configs/ccf_bci/model.yml \
       --model_path output_loss_mixed_1/best_model/model.pdparams \
       --image_path ../data/img_testA \
       --save_dir ../model_result_mixed/

## 模型构建思路及调优过程:
  本项目最先使用Unet作为Baseline进行测试，通过分析预测结果和各模型之间的特点最后选择HRNet作为最终框架，并根据数据标注的物体结构特点选择了CrossEntropyLoss+LovaszSoftmaxLoss+DiceLoss的混合Loss。